In [1]:
from retrieve_data_by_query import RetrieveDataByQuery
t = RetrieveDataByQuery()

In [7]:
query = "Artificial Intelligence"

In [2]:
def get_follower_data(data:list):
    result = []
    for d in data:
        node = d.get("node", {})
        result.append({
            "pk": node.get("pk", ""),
            "username": node.get("username", ""),
            "follower_count": node.get("follower_count", "")
        })
    return result
    
def get_all_follower(userid:str):
    results = []
    response = t.retrieve_follower_by_userid(userid)
    data = response.get("data", {})
    data = data.get("user", {})
    data = data.get("followers",{})
    results.extend(get_follower_data(data.get("edges", [])))
    has_next_page = data.get("page_info", {}).get("has_next_page", False)
    after = data.get("page_info", {}).get("end_cursor")
    
    while has_next_page:
        response = t.retrieve_follower_by_userid_after(userid, after)
        data = response.get("data", {})
        data = data.get("fetch__XDTUserDict", {})
        data = data.get("followers",{})
        results.extend(get_follower_data(data.get("edges", []))) 
        has_next_page = data.get("page_info", {}).get("has_next_page", False)
        
    return results

In [2]:
import json
import pandas as pd

def get_post_data(posts: list):
    results = []
    for post in posts:
        dati = post["node"]["thread"]["thread_items"]
        for p in dati:
            d = p['post']
            if int(d.get("like_count", 0)) < 100:
                continue
            userpk = d.get("user", {}).get("pk", "")
            caption = d.get("caption", {})
            caption_text = caption.get("text", "") if caption else ""
            if caption_text == "":
                continue
            results.append({
                "pk": d.get("pk", ""),
                "user_pk": userpk,
                "caption": caption_text,
                "like_count": int(d.get("like_count", 0)),
                "taken_at": d.get("taken_at", "")
            })
    return results

results = []
response = t.retrieve_thread_by_query(query)
data = response.get("data", {}).get("searchResults", {})
results.extend(get_post_data(data.get("edges", [])))
has_next_page = data.get("page_info", {}).get("has_next_page", False)
after = data.get("page_info", {}).get("end_cursor")
i = 1000

while has_next_page and i > 0:
    response = t.retrieve_thread_by_query_after(query, after)
    data = response.get("data", {}).get("searchResults", {})
    results.extend(get_post_data(data.get("edges", [])))
    has_next_page = data.get("page_info", {}).get("has_next_page", False)
    after = data.get("page_info", {}).get("end_cursor")
    i -= 1

# Creazione del DataFrame
df = pd.DataFrame(results)
df.shape


KeyboardInterrupt



In [20]:
df.drop_duplicates(inplace=True)

In [21]:
df.to_csv("data.csv", index=False)

In [23]:
df['taken_at'] = pd.to_datetime(df['taken_at'], unit='s')

In [3]:
import json
import pandas as pd
df = pd.read_csv("data.csv")

In [4]:
df['user_pk'] = df['user_pk'].astype(str)

In [6]:
unique_user_pk = df['user_pk'].unique()

In [7]:
len(unique_user_pk)
unique_user_pk[0]

'187264532'

In [5]:
get_all_follower("187264532")

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [8]:
import pandas as pd
import concurrent.futures
import os

def fetch_followers(pk):
    follower_data = get_all_follower(pk)
    return [
        {
            "user_pk": pk,
            "follower_pk": follower["pk"],
            "follower_username": follower["username"],
            "follower_count": follower["follower_count"]
        }
        for follower in follower_data
    ]

def load_processed_followers(file_path):
    """Carica i follower già processati se il file esiste."""
    if os.path.exists(file_path):
        df = pd.read_csv(file_path)
        return set(df['follower_pk'].tolist())
    return set()

# Percorso file CSV
csv_file = "followers_data.csv"
processed_followers = load_processed_followers(csv_file)

def save_to_csv(df, file_path):
    """Salva il DataFrame nel CSV senza sovrascrivere quelli esistenti."""
    if os.path.exists(file_path):
        df.to_csv(file_path, mode='a', header=False, index=False)
    else:
        df.to_csv(file_path, index=False)

follower_list = []
batch_size = 1 

with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
    for user_pk in unique_user_pk:
        results = fetch_followers(user_pk)
        
        # Filtra solo i nuovi follower non ancora salvati
        new_followers = [f for f in results if f['follower_pk'] not in processed_followers]
        
        follower_list.extend(new_followers)
        
        if len(follower_list) >= batch_size:
            follower_df = pd.DataFrame(follower_list)
            save_to_csv(follower_df, csv_file)
            processed_followers.update([f['follower_pk'] for f in follower_list])
            follower_list = []  # Svuota la lista per il prossimo batch

# Salva eventuali follower rimasti alla fine
if follower_list:
    follower_df = pd.DataFrame(follower_list)
    save_to_csv(follower_df, csv_file)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
follower_df.to_csv("follower_data.csv", index=False)